In [1]:
from IPython.display import display
import ipywidgets as widgets
import time
import threading

In [2]:
import time
import os
import importlib
from qm import QuantumMachinesManager
import config_qubit as config

2026-01-23 11:01:26,893 - qm - INFO     - Starting session: 892c4f20-f433-44fa-8fe7-081ae895ef82


In [4]:
# Connect to the cluster
import QM_cluster
qmm = QuantumMachinesManager(host=QM_cluster.QM_Router_IP, cluster_name=QM_cluster.cluster_name, log_level="ERROR", octave_calibration_db_path=os.getcwd()) 

In [8]:
def reload_qm(output):
    with output:
        importlib.reload(config)
        qm = qmm.open_qm(config.config)
        print(f'{time.asctime()} QM is ready with id {qm.id}')

def calibrate_qm(output,cal_qubit=True,cal_resonator=True):
    with output:
        importlib.reload(config)
        qm = qmm.open_qm(config.config)
        if cal_qubit:
            caldict = {config.qubit_LO: [config.qubit_IF,]}
            print(f'Calibrating qubit at {config.qubit_LO/1e6:.1f} + {config.qubit_IF/1e6:.1f} MHz')
            qm.calibrate_element('qubit',caldict)
            print('Done')
        if cal_resonator:
            caldict = {config.resonator_LO: [config.resonator_IF,]}
            print(f'Calibrating resonator at {config.resonator_LO/1e6:.1f} + {config.resonator_IF/1e6:.1f} MHz')
            qm.calibrate_element('resonator',caldict)
            print('Done')
        qm = qmm.open_qm(config.config)
        print(f'{time.asctime()} QM is ready with id {qm.id}')

class QueueMonitor(threading.Thread):
    def __init__(self, output, QM_label, job_table):
        super().__init__()
        self.output = output
        self.job_table = job_table
        self.QM_label = QM_label
        self.keeprunning = True
    def run(self):
        while self.keeprunning:
            self.parse_queue()
            time.sleep(0.1)
        self.output.append_stdout("Done\n")
    def stop(self):
        self.keeprunning = False
    def kill(self):
        try:
            qm_list = qmm.list_open_qms()
            qm = qmm.get_qm(qm_list[0])
        except:
            qm = None
        if qm:
            job = qm.get_running_job()
            if job:
                job.halt()
    def parse_queue(self):
        try:
            qm_list = qmm.list_open_qms()
            qm = qmm.get_qm(qm_list[0])
        except:
            qm = None
        if qm:
            try:
                self.QM_label.value = f"Jobs on {qm.id}"
                table = []
                for job in qm.queue.pending_jobs:
                    table.append(f"""<tr><td>Pending</td><td>{job.id}</td></tr>""")
                job = qm.get_running_job()
                if job:
                    table.append(f"""<tr><td>Running</td><td>{job.id}</td></tr>""")
                rows = " ".join(table)
                self.job_table.value = f"<table>{rows}</table>"
            except:
                self.job_table.value = "<p>Error while parsing the queue</p>"
        else:
            self.QM_label.value = "No QM running"

# Load QM

In [ ]:
# Load config into the QM
importlib.reload(config)
qm = qmm.open_qm(config.config)
qm.id = time.time()
print(f'{time.asctime()} QM is ready with id {qm.id}')

In [18]:
# Interactive reload and calibrate
button_1 = widgets.Button(description="Reload QM")
button_2 = widgets.Button(description="Calibrate QM")
output = widgets.Output()

display(button_1, button_2, output)
        
button_1.on_click(lambda button: reload_qm(output))
button_2.on_click(lambda button: calibrate_qm(output,True,True))

Button(description='Reload QM', style=ButtonStyle())

Button(description='Calibrate QM', style=ButtonStyle())

Output()

# Monitor QM

In [12]:
# Queue monitoring
button_stop = widgets.Button(description='Stop')
button_kill = widgets.Button(description='Kill')
QM_label = widgets.Label(value="")
job_table = widgets.HTML(value="")
output = widgets.Output()
monitor = QueueMonitor(output,QM_label,job_table) 
monitor.start()
display(widgets.HBox([button_stop, button_kill]), QM_label, job_table, output)
button_stop.on_click(lambda button: monitor.stop())
button_kill.on_click(lambda button: monitor.kill())

Label(value='')

HTML(value='')

Output()